In [36]:
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

In [2]:

def ordinal_crossentropy(num_classes=5):
    # Weight matrix: squared distance penalty
    weights = tf.constant([[ (i - j)**2 for j in range(num_classes)] for i in range(num_classes)], dtype=tf.float32)
    def loss_fn(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        true_idx = tf.argmax(y_true, axis=-1)
        weight_row = tf.gather(weights, true_idx)
        # penalize log-probabilities by squared distance
        loss = -tf.reduce_sum(weight_row * tf.math.log(y_pred), axis=-1)
        return tf.reduce_mean(loss)
    return loss_fn


def mae_level(y_true, y_pred):
    # mean absolute error on the expected class level
    num_classes = tf.shape(y_pred)[-1]
    levels = tf.cast(tf.range(num_classes), tf.float32)
    pred_level = tf.reduce_sum(y_pred * levels, axis=-1)
    true_level = tf.cast(tf.argmax(y_true, axis=-1), tf.float32)
    return tf.reduce_mean(tf.abs(pred_level - true_level))

In [25]:

def build_model(input_shape=(256,256,3), num_classes=5):
    base = EfficientNetB0(include_top=False, weights=None, input_shape=input_shape)
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base.input, outputs=outputs)

In [ ]:
# train_dir = 'train_preprocessed_augmented'
# val_dir   = 'validation_preprocessed'
# train_csv = 'augmented_labels.csv'
# val_csv   = 'trainLabels/validation_updated.csv'

In [ ]:
# train_df = pd.read_csv(train_csv)
# val_df   = pd.read_csv(val_csv)

In [ ]:
# train_labels = tf.keras.utils.to_categorical(train_df['level'], num_classes=5)
# val_labels   = tf.keras.utils.to_categorical(val_df['level'], num_classes=5)

In [10]:
def load_image(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (256,256))
        img = tf.cast(img, tf.float32) / 255.0
        return img, label

In [11]:
def make_dataset(df, img_dir, labels, batch_size=16, shuffle=True):
    paths = [os.path.join(img_dir, f + '.jpeg') for f in df['image']]
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1000)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# train_ds = make_dataset(train_df, train_dir, train_labels, batch_size=16, shuffle=True)
# val_ds   = make_dataset(val_df, val_dir,   val_labels,   batch_size=16, shuffle=False)

In [37]:
img_dir = 'train'               # folder with all images
labels_csv = 'trainLabels/train_updated.csv'       # CSV with columns: image,level
test_size = 0.2                 # 20% for validation
random_state = 42               # for reproducibility
batch_size = 16

    # Load full labels
df_all = pd.read_csv(labels_csv)
    # Stratified split into train/validation
train_df, val_df = train_test_split(
        df_all,
        test_size=test_size,
        stratify=df_all['level'],
        random_state=random_state
    )


In [38]:
num_classes = 5
train_labels = tf.keras.utils.to_categorical(train_df['level'], num_classes)
val_labels   = tf.keras.utils.to_categorical(val_df['level'],   num_classes)

    # Build datasets
train_ds = make_dataset(train_df, img_dir, train_labels, batch_size=batch_size, shuffle=True)
val_ds   = make_dataset(val_df,   img_dir, val_labels,   batch_size=batch_size, shuffle=False)


In [ ]:
 # Build & compile
model = build_model()
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=ordinal_crossentropy(),
        metrics=['categorical_accuracy', mae_level]
    )

    # Callbacks
callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
        ModelCheckpoint('best_ordinal_model.keras', monitor='val_loss', save_best_only=True),
        CSVLogger('ordinal_training_log.csv')
    ]
model.summary()

In [ ]:
    # Train
model.fit(
        train_ds,
        epochs=25,
        validation_data=val_ds,
        callbacks=callbacks
    )

model.save('ordinal_regression_model.keras')


Epoch 1/25
1706/1707 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - categorical_accuracy: 0.1308 - loss: 47.8743 - mae_level: 1.9759

2025-08-08 06:17:29.618908: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-08 06:17:29.726756: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


1707/1707 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - categorical_accuracy: 0.1308 - loss: 47.8710 - mae_level: 1.9761

2025-08-08 06:18:40.870788: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-08 06:18:40.999398: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


1707/1707 ━━━━━━━━━━━━━━━━━━━━ 313s 154ms/step - categorical_accuracy: 0.0801 - loss: 42.1216 - mae_level: 2.2208 - val_categorical_accuracy: 0.0149 - val_loss: 33.3456 - val_mae_level: 2.6557 - learning_rate: 1.0000e-04
Epoch 2/25
1707/1707 ━━━━━━━━━━━━━━━━━━━━ 227s 129ms/step - categorical_accuracy: 0.0208 - loss: 35.2012 - mae_level: 2.6568 - val_categorical_accuracy: 0.0149 - val_loss: 33.5042 - val_mae_level: 2.7458 - learning_rate: 1.0000e-04
Epoch 3/25
1707/1707 ━━━━━━━━━━━━━━━━━━━━ 228s 130ms/step - categorical_accuracy: 0.0177 - loss: 33.9615 - mae_level: 2.7049 - val_categorical_accuracy: 0.0185 - val_loss: 32.5804 - val_mae_level: 2.7442 - learning_rate: 1.0000e-04
Epoch 4/25
1707/1707 ━━━━━━━━━━━━━━━━━━━━ 228s 130ms/step - categorical_accuracy: 0.0180 - loss: 33.4821 - mae_level: 2.7080 - val_categorical_accuracy: 0.0149 - val_loss: 33.2002 - val_mae_level: 2.7245 - learning_rate: 1.0000e-04
Epoch 5/25
1707/1707 ━━━━━━━━━━━━━━━━━━━━ 231s 132ms/step - categorical_accuracy: 0

KeyboardInterrupt: 

: 

In [30]:
y_pred_probs = model.predict(val_ds)


63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 95ms/step


In [31]:
y_pred_probs

array([[0.02971547, 0.03095572, 0.10169767, 0.3063826 , 0.53124857],
       [0.02929074, 0.03048587, 0.1043134 , 0.30147937, 0.5344306 ],
       [0.0283641 , 0.03034564, 0.10458735, 0.29874676, 0.5379561 ],
       ...,
       [0.03033291, 0.03126829, 0.10119895, 0.30849192, 0.528708  ],
       [0.02804465, 0.03070766, 0.10824221, 0.300292  , 0.5327135 ],
       [0.03545425, 0.04295755, 0.13712889, 0.29873002, 0.48572922]],
      dtype=float32)

In [35]:

y_true=val_df['level'].values
for i in range(100, 110):
    print(f"True: {y_true[i]} | Raw Predicted: {y_pred_probs[i]} ")

True: 0 | Raw Predicted: [0.02891908 0.03101681 0.1057616  0.3029802  0.53132224] 
True: 0 | Raw Predicted: [0.02872338 0.03018536 0.10549872 0.30075872 0.53483385] 
True: 0 | Raw Predicted: [0.02829686 0.03047849 0.10525917 0.30215007 0.53381544] 
True: 0 | Raw Predicted: [0.02918054 0.029815   0.09845775 0.3079098  0.5346369 ] 
True: 0 | Raw Predicted: [0.02844117 0.03103023 0.10946763 0.29933327 0.5317277 ] 
True: 0 | Raw Predicted: [0.02843874 0.03130361 0.10921185 0.29842365 0.5326222 ] 
True: 0 | Raw Predicted: [0.02869823 0.03036305 0.10347905 0.30397442 0.5334851 ] 
True: 0 | Raw Predicted: [0.0357343  0.04449081 0.12912875 0.29860407 0.4920421 ] 
True: 0 | Raw Predicted: [0.02920434 0.03017196 0.1019477  0.30653933 0.53213674] 
True: 0 | Raw Predicted: [0.02903535 0.0306626  0.10195847 0.30609474 0.53224885] 
